In [ ]:
## Import necessary libraries (Pytorch)
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms


In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input - 28x28X1 #output - 28x28x32 RF- 3x3
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1) #input - 28x28x32 output-28x28x64 RF-5x5
        self.pool1 = nn.MaxPool2d(2, 2) # input - 28x28x64 output-14x14x64 RF-10x10
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1) # input - 14x14x64 output-14x14x128 RF-12x12
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1) # input - 14x14x128 output-14x14x256 RF-14x14
        self.pool2 = nn.MaxPool2d(2, 2) # input - 14x14x256 output-7x7x256 RF-28x28
        self.conv5 = nn.Conv2d(256, 512, 3) # input - 7x7x256 output-5x5x512 RF-30x30
        self.conv6 = nn.Conv2d(512, 1024, 3) # input - 5x5x512 output-3x3x1024 RF-32x32
        self.conv7 = nn.Conv2d(1024, 10, 3) # input - 3x3x1024 output-1x1x10 RF-34x34
        #self.fc1 = nn.Linear(10, 10) #input - 1x1x10 output - 1x10

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x))))) # input image -> 1st Conv -> relu activation -> 2nd Conv -> relu activation -> max pooling -> 
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x))))) # 3rd Conv -> relu -> 4th layer of Conv -> relu -> Max pooling ->
        x = F.relu(self.conv6(F.relu(self.conv5(x)))) # 5th Conv -> relu -> 6th Conv -> relu ->
        #x = F.relu(self.conv7(x)) # 7th layer Conv -> Relu
        x=self.conv7(x)
        x = x.view(-1, 10) #Reshape to 1d array with 10 cells
       # x = self.fc1(x)
        return F.log_softmax(x) #return the output after log(Softmax(x)) function

In [ ]:
!pip install torchsummary # install torchsummary package
from torchsummary import summary #Import summary function from torchsummary library
use_cuda = torch.cuda.is_available() # Check if GPU (Cuda) is available
device = torch.device("cuda" if use_cuda else "cpu") # Assign GPU to device if available else assign CPU
model = Net().to(device)  #move the model to GPU or CPU
summary(model, input_size=(1, 28, 28)) # Print the model architecture summary

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Conv2d-9             [-1, 10, 1, 1]          92,170
Total params: 6,379,786
Trainable params: 6,379,786
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.51
Params size (MB): 24.34
Estimated Total Size (MB): 25.85
-------------------------------------

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:
use_cuda

True

In [ ]:

torch.manual_seed(1) #Set random seed
batch_size = 128 # Batch size 128 which means 128 images are taken for traning simultaneously

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {} # pin memory copies data to pinned memory, num_workers assign parallel processing for GPU
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([ # use multiple transform together
                        transforms.ToTensor(), # convert image to tensor type
                        transforms.Normalize((0.1307,), (0.3081,)) # normalize tensor image with mean of 0.1307 & std  0.3081
                    ])),  
    batch_size=batch_size, shuffle=True, **kwargs) #Load MNIST train data as torch tensor 
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs) #Load MNIST test data as torch tensor


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [ ]:
from tqdm import tqdm # Import tqdm for progress bar for iterations
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar): # Iterate over input traing image for trainging model
        data, target = data.to(device), target.to(device) # Send data to GPU
        optimizer.zero_grad() # Clears x.grad for this iterations ( Intializes gradients as 0)
        output = model(data) # fit and train the nn model on the data 
        loss = F.nll_loss(output, target) # Compute negative log likelihood loss function 
        loss.backward() # Compute gradients for backpropogation ( gradients are stored internally in x.grad )
        optimizer.step() # use the internally stored gradients to updates all parameters based on optimizer
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}') # print loss value and Batch id


def test(model, device, test_loader):
    model.eval() #intialise model validation 
    test_loss = 0 
    correct = 0
    with torch.no_grad(): # no grade return true if gradients are not computed which happens only during train , not  during valdiation
        for data, target in test_loader:
            data, target = data.to(device), target.to(device) # Send data to GPU
            output = model(data) # determine output for the test image batch
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item() # compare prediction with target image return 1 if matching

    test_loss /= len(test_loader.dataset) # Calculate avg test loss

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset))) # print Test loss, average loss and Accuracy of the model

In [ ]:

model = Net().to(device) # Create object of Net class and send to the device (GPU)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9) # initialize Stochastic Gradient decent for gradient optimizer

for epoch in range(1, 2): # no of epochs is 1
    train(model, device, train_loader, optimizer, epoch) # Call train fucntion 
    test(model, device, test_loader) # call test function

loss=0.033061008900403976 batch_id=468: 100%|██████████| 469/469 [00:18<00:00, 25.03it/s]



Test set: Average loss: 0.0654, Accuracy: 9800/10000 (98%)

